In [20]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings("ignore")
import matplotlib.pyplot as plt
from scipy.special import comb
from scipy.stats import binom

In [2]:
df_case_js=pd.read_excel('datafile/casedata.xlsx')
df_case=pd.read_csv('datafile/case_time.csv')
df_test=pd.read_csv('datafile/testdata.csv')
df_nsw=pd.read_csv('datafile/COVID_AU_state_cumulative.csv')

In [3]:
df_case_js.drop(columns=['Venue','Address','Time','Alert','HealthinformationHTML'], inplace=True)

In [4]:
df_case.fillna(method='ffill',inplace=True)
df_case['postcode']=df_case.postcode.astype('int')
df_case.drop(columns=['lga_name19','lga_code19','lhd_2010_code','lhd_2010_name'], inplace=True)
df_case.rename(columns={'notification_date':'date'},inplace=True)
df_test.drop(columns=['lga_name19','lga_code19','lhd_2010_code','lhd_2010_name'], inplace=True)
df_test.rename(columns={'test_date':'date'},inplace=True)
df_case['infected']=1
dfx=df_case.loc[df_case['postcode']==2026]
dfx['date']=pd.to_datetime(dfx['date'])
dfx.drop(columns='postcode',inplace=True)
z=dfx.groupby('date')['infected'].sum().reset_index() # infected
x=df_test.loc[df_test['postcode']==2026]
x['date']=pd.to_datetime(x['date'])
rng = pd.date_range('2020-1-30', periods=100, freq='D')
df = pd.DataFrame({ 'date': rng }) 
m = x.merge(df, left_on=['date'], right_on=['date'], how='right')
d = z.merge(m, left_on=['date'], right_on=['date'], how='right') 
d=d.fillna(0)
d.drop(columns='postcode',inplace=True)
df_nsw=df_nsw.loc[df_nsw['state_abbrev']=='NSW']
df_nsw.drop(columns=['state', 'state_abbrev'],inplace =True)
df_nsw['date']=pd.to_datetime(df_nsw['date'])
final = df_nsw.merge(d, left_on=['date'], right_on=['date'], how='right')
final.set_index(final['date'],inplace=True)
final.drop(columns='date',inplace=True)
final=final.astype('int')

In [13]:
final.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 100 entries, 2020-01-30 to 2020-05-08
Data columns (total 10 columns):
 #   Column      Non-Null Count  Dtype
---  ------      --------------  -----
 0   confirmed   100 non-null    int32
 1   deaths      100 non-null    int32
 2   tests       100 non-null    int32
 3   positives   100 non-null    int32
 4   recovered   100 non-null    int32
 5   hosp        100 non-null    int32
 6   icu         100 non-null    int32
 7   vent        100 non-null    int32
 8   infected    100 non-null    int32
 9   test_count  100 non-null    int32
dtypes: int32(10)
memory usage: 4.7 KB


# UA: 

- Gọi: 
   + n: là dân số (population)
   + c(t) là tổng số ca nhiễm theo t
   + q là tỉ lệ người UA bị bệnh, T là số người tham gia quá trình detection (có thể coi là số người admit to clinic trong NSW dataset)
   + T là thời gian phát hiện bệnh. Trong đây coi T là 10 ngày (incubation period) nhưng lấy trung bình là 7.5 ngày
   + k là size of sample
   + m is the number of people infected in the size k

In [8]:
#defined subjects: 
N = 25e6 #population
# q: fraction of underascertainment
c_array=final['confirmed'] # culmulative cases
T=8 #incubation period


In [19]:
com(k,m)

21.0

In [10]:
from sympy import *
from scipy.optimize import fsolve

from sympy.solvers.solveset import linsolve

In [2]:
a,b= symbols('a, b')

In [3]:
linsolve([a+b- 1, a-b - 3 ], (a,b))

FiniteSet((2, -1))

In [11]:
fsolve([b+ 1-exp(-a*2), 2*b+ log(1-a)],(a,b))

TypeError: 'list' object is not callable